# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686594


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:38,  3.39s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:08,  2.44s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:45,  1.75s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:31,  1.28s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  6.99it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  7.02it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  5.92it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  6.49it/s]

not-logged-in-3b85ef6ad3e2968c6631    0.012378
not-logged-in-20a275b53abec8db9f41    0.862350
richiemac                             0.000134
not-logged-in-a9247261537fa1da33fe    0.007678
cschwefl                              0.000130
Dynamatt                              0.000163
kjkavene                              0.000122
jmfranci                              0.000576
tosnyder                              0.000250
ncaro                                 0.000412
aistatic                              0.091485
Alexatwin                             0.000671
Pooki748                              0.037157
FelipedaFonseca                       0.000470
not-logged-in-41e9fe197dc7a2be8424    0.000108
not-logged-in-beca27757714dcf5eff9    0.011226
pangeli5                              0.000235
Lavadude                              0.078540
ShirleyL                              0.000112
harmonioushope                        0.000130
w7250369                              0.001542
r.sundstrom3 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())